In [1]:
#hiddencell
from pbl_tools import *

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(fname = 'NotoSansKR-Regular.otf', name = 'NotoSansKR')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='NotoSansKR')

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# 스테이지 6. test 데이터 전처리 및 모델 학습과 평가

## 도입 

이번 스테이지에서는 'test 데이터 전처리 및 모델 학습'에 중점을 두고 진행됩니다. 
이전 스테이지에서는 데이터의 기본적인 구조와 통계량을 확인하였으며,   
이제는 테스트 데이터를 다루며 데이터 전처리와 모델 학습을 통해 예측 모델을 고도화하고자 합니다.

## 학습 목표  
- 테스트 데이터에 대한 전처리를 수행하고 필요한 새로운 특성을 생성할 수 있다.
- 독립 변수와 종속 변수를 명확히 분리할 수 있다.
- PCA (주성분 분석) 모델을 활용하여 데이터의 차원을 축소하고 주요 특성을 추출할 수 있다.
- GradientBoostingClassifier 모델을 활용하여 테스트 데이터를 분석하고 모델 학습을 진행할 수 있다.

# 1. 데이터 전처리 및 새로운 특성 생성
[문제 1]  
- education 피처의 `Some-college, Assoc-acdm, Assoc-voc` 값들을 `SomeHigherEd`로 재구분해주세요. 이를 원래의 데이터프레임에 적용될 수 있도록 해주세요.
- marital.status 피처의 `'Separated', 'Divorced'` 값들을 `MarriageEnded` 로 재구분해주세요. 이를 원래의 데이터프레임에 적용될 수 있도록 해주세요.
- `age` 피처와 `hours.per.week` 피처를 곱한 `age-hours` 파생변수를 생성해 주세요.


In [ ]:
import pandas as pd  

train = pd.read_csv('train.csv')  
test = pd.read_csv('test.csv')  

# 교육(education) 수준 카테고리 재분류
train['education'].replace(['Preschool', '10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th'], 'LowEducation', inplace=True)
train['education'].replace([___, ___, ___], ___, inplace=___)
train['education'].replace(['Masters', 'Prof-school'], ['Masters', 'Masters'], inplace=True)

#결혼 상태 (marital.status) 수준 카테고리 재분류
train['marital.status'].replace(['Never.married', 'Married.spouse.absent'], 'UnmarriedStatus', inplace=True)
train['marital.status'].replace(['Married.AF.spouse', 'Married.civ.spouse'], 'Married', inplace=True)
train['marital.status'].replace(...)

# 나이(age)와 주당 근무시간(hours.per.week)을 곱한 새로운 특성 생성
train['age-hours'] = train[___] * train[___]

In [3]:
#checkcode
ensure_vals(globals(), 'train')
@check_safety
def check(
    df: pd.DataFrame,
    col: str,
    val1: str,
    val2: str,
    val3: str,
    new_col: str
):
    c_point0 = hasattr(df, 'head')
    c_point1 = val1 in df[col].unique()
    c_point2 = val2 in df[col].unique()
    c_point3 = val3 not in df[col].unique()
    c_point4 = new_col in df.columns

    if (
        c_point0 and 
        c_point1 and 
        c_point2 and 
        c_point3 and 
        c_point4
    ):
        return True
    else:
        return False

check(train, 'education', 'LowEducation', 'SomeHigherEd', 'Prof-school', 'age-hours')

True

# 2.test 데이터에 대한 전처리 및 새로운 특성 생성

train 데이터셋에 적용한 데이터 전처리를 test 데이터셋에도 동일하게 적용하여 일관성을 유지하고 모델의 일반화 성능을 향상시킵니다.

In [4]:
# 교육(education) 수준 카테고리 재분류
test['education'].replace(['Preschool', '10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th'], 'LowEducation', inplace=True)
test['education'].replace(['Some-college', 'Assoc-acdm', 'Assoc-voc'], 'SomeHigherEd', inplace=True)
test['education'].replace(['Masters', 'Prof-school'], ['Masters', 'Masters'], inplace=True)

#결혼 상태 (marital.status) 수준 카테고리 재분류
test['marital.status'].replace(['Never.married', 'Married.spouse.absent'], 'UnmarriedStatus', inplace=True)
test['marital.status'].replace(['Married.AF.spouse', 'Married.civ.spouse'], 'Married', inplace=True)
test['marital.status'].replace(['Separated', 'Divorced'], 'MarriageEnded', inplace=True)

# 나이(age)와 주당 근무시간(hours.per.week)을 곱한 새로운 특성 생성
test['age-hours'] = test['age']*test['hours.per.week']

# 3.결측치 처리와 불필요한 열 제거

[문제 2]
- 결측치 처리를 위한 `SimpleImputer` 클래스를 가져오세요.    
- `SimpleImputer` 객체를 생성하고, 결측치를 처리할 때 해당 특성에서 가장 자주 발생하는 값으로 결측치를 대체해주세요.
- 학습 데이터셋인 `train`의 'occupation'과 'workclass' 열에 대해 결측치를 `최빈값`으로 채워주세요.
- 테스트 데이터셋인 `test`의 'occupation'과 'workclass' 열에 대해 결측치를 `최빈값`으로 채워주세요.
- train, test 데이터셋에서 불필요한 피처인 `'native.country','ID'` 을 제거해 주세요.

In [297]:
from sklearn.impute import ___

# SimpleImputer를 사용하여 결측치를 최빈값으로 보간
imputer = ___(strategy='___')
train[['occupation','workclass']] = ___.___(...)
test[['occupation','workclass']] = ___.___(...)

# 불필요한 열 제거
train = train.___(...)
test = test.___(...)

In [6]:
#checkcode
ensure_vals(globals(), 'x_train', 'imputer')
@check_safety
def check(
    df: pd.DataFrame,
    encoder: SimpleImputer,
    not_col: str,
    use_col1: str,
    use_col2: str
):
    c_point0 = not_col not in df.columns
    c_point1 = use_col1 in encoder.feature_names_in_
    c_point1 = use_col2 in encoder.feature_names_in_

    if c_point0 and c_point1:
        return True
    else:
        return False

check(train, imputer, 'native.country', 'workclass', 'occupation')

True

# 4.원-핫 인코딩을 사용한 범주형 변수 변환

[문제 3]
- scikit-learn의 preprocessing 모듈에서 `OneHotEncoder` 클래스를 불러오세요.
- `OneHotEncoder` 객체를 생성합니다. 
- 학습 데이터셋(`train`)의 `'race', 'sex', 'marital.status'` 열을 선택하고, 원-핫 인코딩을 적용해주세요.
- 테스트 데이터셋(`test`)의 `'race', 'sex', 'marital.status'` 열을 선택하고, 원-핫 인코딩을 적용해주세요.
- 원-핫 인코딩된 훈련 데이터를 새로운 데이터프레임으로 변환해주세요.   
컬럼 이름은 'race_xxx', 'sex_xxx', 'marital.status_xxx' 형태의 이름으로 생성해주세요. 여기서 xxx는 해당 카테고리 변수의 각 범주를 나타냅니다.
- test 데이터셋 기존의 인덱스를 삭제하고 새로운 연속적인 정수 인덱스로 설정해주세요.
- test와 test_ohe를 열 방향(axis=1)으로 합쳐주세요.

In [ ]:
from sklearn.preprocessing import ___

encoder = ___(sparse_output=False)
train_encoded = ___.fit_transform(train[['race', 'sex', 'marital.status']])
test_encoded = ___.___(...)

# 원-핫 인코딩된 데이터를 DataFrame으로 변환
train_ohe = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(['race', 'sex', 'marital.status']))
test_ohe = pd.DataFrame(___, columns=___.___(...))

# 데이터프레임에 원-핫 인코딩된 특성 추가
train = train.reset_index(drop=True)
test = test.___(___=___)

train = pd.concat([train,train_ohe], axis=1)
test = pd.concat(...)

In [25]:
#checkcode
ensure_vals(globals(), 'train', 'test')
@check_safety
def check(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    enc: OneHotEncoder,
    enc_name: str,
    num_enc_feature: int,
    len_train_col: int,
    len_test_col: int
):
    
    c_point0 = enc_name in str(enc)
    c_point1 = len(enc.feature_names_in_) == num_enc_feature
    c_point2 = len(train_df.columns) == len_train_col
    c_point3 = len(test_df.columns) == len_test_col
    

    if (
        c_point0 and 
        c_point1 and 
        c_point2 and 
        c_point3
    ):
        return True
    else:
        return False

check(train, test, encoder, 'OneHotEncoder', 3, 28, 27)

True

# 5.라벨 인코딩을 사용한 범주형 변수 변환

[문제 4]   
- `LabelEncoder` 객체를 생성하여 le 변수에 할당해 주세요.
- train 의 현재 열의 타입이 object 인 경우 아래 셀을 실행해 주세요.
- `train` 데이터프레임의 현재 열에 LabelEncoder을 적용해 주세요.
- 만약 검증 데이터에서 새롭게 나타나는 범주값(label)이라면, 이를 인코더의 클래스 목록(le.classes_)에 추가해 주세요.
- `test` 데이터프레임의 현재 열에 LabelEncoder을 적용해 주세요.

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

for col in train.columns:
    if train[col].___ == '___':
        
        le = ___()
        train[col] = ___.___(___[___])

        for label in np.unique(test[col]):
            if label not in le.classes_:
                le.classes_ = np.___(le.classes_, ___)
        test[col] = ___.___(___[___])

In [33]:
#checkcode
ensure_vals(globals(), 'train', 'test')
@check_safety
def check(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    obj: str,
    zero: 0,
    enc_name: str
):
    
    len_obj_train = len(train_df.select_dtypes(include=obj).columns)
    len_obj_test = len(test_df.select_dtypes(include=obj).columns)
    
    c_point0 = len_obj_train == zero
    c_point1 = len_obj_test == zero
    c_point2 = enc_name in str(le)

    if c_point0:
        return True
    else:
        return False

check(train, test, 'object', 0, 'LabelEncoder')

True

# 6.StandardScaler를 이용한 데이터 표준화

[문제 5]

- StandardScaler() 객체를 생성해 주세요.
- train 데이터셋에 해당 피처 부분에 표준화를 적용해주세요.
- test 데이터셋에 해당 피처 부분에 표준화를 적용해주세요.


In [ ]:
from sklearn.preprocessing import StandardScaler

features = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss','hours.per.week', 'age-hours']

scaler = ___()
train[features] = ...
test[features] = ...

In [42]:
#checkcode
ensure_vals(globals(), 'train', 'test', 'scaler')
@check_safety
def check(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    scaler_class: StandardScaler,
    num_features: int,
    col1: str,
    col2: str
):
    
    c_point0 = len(scaler_class.feature_names_in_) == num_features
    c_point1 = train_df[col1].dtypes != int
    c_point2 = test_df[col2].dtypes != int

    if c_point0 and c_point1 and c_point2:
        return True
    else:
        return False

check(train, test, scaler, 7, 'age', 'education.num')

True

# 7.독립 변수와 종속 변수 분리

In [43]:
train_x = train.drop(['target'],axis = 1)
train_y = train['target']

# 8.데이터 차원 축소를 위한 PCA 모델 학습

[문제 6]  
- train_x 에 대해 PCA 모델을 학습시켜보세요. 
- `explained_variance_ratio_` 값을 누적해서 더해주세요.
- 해당 임계값을 넘는 최초의 인덱스를 추출해 보세요.
- 주성분 분석(PCA) 모델을 초기화해주세요. 주성분 개수를 전달해 주세요.
- 초기화된 PCA 모델을 기존의 train_x 데이터에 다시 적용해주세요.테스트 데이터인 `test`에 동일한 PCA 모델을 적용합니다.

In [302]:
from sklearn.decomposition import PCA

# PCA 모델 학습
pca = PCA()
pca.___(___)

# 분산의 설명량 확인
explained_variance_ratio = pca.explained_variance_ratio_

# 누적 설명량과 주성분 개수 선택
cumulative_explained_variance = ___.___(explained_variance_ratio)
n_components = ___.___(___ >= 0.95) + 1

# PCA 모델 재설정 및 데이터 변환
pca = ___(n_components=___)

train_x = pca.___(train_x)
test = pca.___(test)

In [66]:
#checkcode
import numpy as np
ensure_vals(globals(), 'train_x', 'test','pca')
@check_safety
def check(
    train_array: np.array,
    test_array: np.array,
    pca_class: PCA,
    indexing: int
):
    
    c_point0 = len(pca_class.get_feature_names_out()) == train_array.shape[1]
    c_point1 = len(pca_class.get_feature_names_out()) == test_array.shape[1]
    
    if c_point0 and c_point1:
        return True
    else:
        return False

check(train_x, test, pca, 1)

True

# 9.GradientBoostingClassifier 모델을 활용한 데이터 분석 및 성능 평가


[문제 7]

- GradientBoostingClassifier 모델을 생성해 주세요. 
- 학습 데이터를 사용하여 학습시켜주세요.
- 학습된 모델을 사용하여 테스트 데이터에 대한 예측을 수행합니다.

In [303]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score

gb_classifier = ___(random_state=30, 
                    max_depth=6,
                    min_samples_split=3,
                    max_features=10)

gb_classifier.___(___, ___)

y_pred = gb_classifier.___(___)

In [81]:
#checkcode
import numpy as np
ensure_vals(globals(), 'gb_classifier', '')
@check_safety
def check(
    model_name: str,
    model: GradientBoostingClassifier,
    train_array: np.ndarray,
    one: int,
    pred_value: np.ndarray,
    pred_shape: tuple
):
    
    c_point0 = model_name in str(model)
    c_point1 = model.n_features_in_ == (train_array.shape[one])
    c_point2 = pred_value.shape == pred_shape
    
    if c_point0 and c_point1 and c_point2:
        return True
    else:
        return False

check('GradientBoostingClassifier', gb_classifier, train_x, 1, y_pred,(494,))

True

# 10.데이터프레임(DataFrame)을 CSV 파일로 저장하기

In [304]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = y_pred
submission.to_csv('submission.csv', index=False)